# idact - Prometheus sandbox

## Initial setup

Add `idact` to path:

In [1]:
import sys
import os
import bitmath
import logging
import subprocess

sys.path.append('../')  # For running in repo. Unnecessary with pip install.
from idact import *

Hide debug information, setup context manager stack (for testing purposes)

## Add cluster (only first run)

If you need a new SSH key to connect to the cluster, specify its type:

In [2]:
key = KeyType.RSA  # Generate a new RSA key (Default location: ~/.ssh)

If you already have a key, uncomment this and provide its absolute path:

In [3]:
# key = os.path.expanduser('~/.ssh/id_rsa')

If you set `install_key` to False, you will not be asked for a password later, but the key must be installed on the cluster manually.

In [4]:
install_key = True

Add cluster:

In [5]:
cluster = add_cluster(name="pro",
                      user="plggarstka",
                      host="pro.cyfronet.pl",
                      port=22,
                      auth=AuthMethod.PUBLIC_KEY,
                      key=key,
                      install_key=install_key,
                      scratch="$SCRATCH")
save_environment()

2018-11-15 00:20:01 INFO: Generating public-private key pair.


## Load cluster (subsequent runs)

In [6]:
load_environment()
cluster = show_cluster("pro")
cluster

Cluster(pro.cyfronet.pl, 22, plggarstka, auth=AuthMethod.PUBLIC_KEY, key='C:\\Users\\Maciej/.ssh\\id_rsa_j7', install_key=True, disable_sshd=False)

Debug log is saved to `idact.log` for every session if you need to troubleshoot or report bugs, but you can also change log level for messages printed to standard output:

In [7]:
set_log_level(logging.INFO)
save_environment()

In [8]:
node = cluster.get_access_node()
node

Node(pro.cyfronet.pl:22, None)

On your first action, you may be asked for a password to install the key, if you chosen `install_key=True` while adding the cluster.
You can connect explicitly to do this right now:

In [9]:
node.connect()

2018-11-15 00:20:07 INFO: Installing key using password authentication.
Password for plggarstka@pro.cyfronet.pl:22: 


It's important to save the environment installing the key, so it's installed only once:

In [10]:
print(cluster.config.install_key)
save_environment()  # Never install the key again.

False


You can run commands on the login node now:

In [11]:
node.run('whoami')

'plggarstka'

In [12]:
node.run('hostname')

'login01.pro.cyfronet.pl'

## Allocate nodes

In [13]:
nodes = cluster.allocate_nodes(nodes=2,
                               cores=2,
                               memory_per_node=bitmath.GiB(10),
                               walltime=Walltime(minutes=20),
                               native_args={
                                   '--partition': 'plgrid-testing',
                                   '--account': 'intdata'
                               })

2018-11-15 00:20:24 INFO: Creating the ssh directory.


In [14]:
nodes

Nodes([Node(NotAllocated),Node(NotAllocated)], SlurmAllocation(job_id=14194860))

In [15]:
nodes.wait()
nodes

Nodes([Node(p0097:54199, 2018-11-14 23:40:31.762414+00:00),Node(p0100:33033, 2018-11-14 23:40:31.762414+00:00)], SlurmAllocation(job_id=14194860))

## Run commands

In [16]:
nodes[0].run('whoami')

'plggarstka'

In [17]:
nodes[0].run('hostname')

'p0097'

In [18]:
nodes[1].run('squeue')

'JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)\n          14194860 plgrid-te     wrap plggarst  R       0:07      2 p[0097,0100]'

In [19]:
nodes[1].run('hostname')

'p0100'

## Examine node resources

In [20]:
nodes[0].resources.memory_total

GiB(10.0)

In [21]:
nodes[0].resources.memory_usage

GiB(0.0225067138671875)

In [22]:
nodes[0].resources.cpu_cores

2

In [23]:
nodes[0].resources.cpu_usage

1.0

## Tunnel

In [24]:
tunnel = nodes[0].tunnel(here=9000, there=10000)

In [25]:
tunnel

MultiHopTunnel(9000:10000)

In [26]:
tunnel.close()

## Deploy notebook

You need to provide Bash commands that will expose a Python distribution you want work with on the cluster. It should have `idact` installed. Installing `idact` with pip will also install the required `jupyter` package.

In [27]:
cluster.config.setup_actions.jupyter = ['module load plgrid/tools/python-intel/3.6.2']
save_environment()

To run Jupyter Notebook on the cluster:

In [28]:
nb = nodes[0].deploy_notebook(local_port=8080)
nb

JupyterDeployment(8080 -> Node(p0097:54199, 2018-11-14 23:40:31.762414+00:00)

In [29]:
nodes[0].resources.memory_usage

GiB(0.08160018920898438)

In [30]:
nb.local_port

8080

To open the deployed notebook server in a new tab:

In [31]:
nb.open_in_browser()

In [32]:
nodes[0].resources.memory_usage

GiB(0.08343124389648438)

### Push and pull notebook

You can access the deployed notebook from multiple places by first pushing it:

In [33]:
cluster.push_deployment(nb)

2018-11-15 00:21:06 INFO: Pushing deployment: JupyterDeployment(8080 -> Node(p0097:54199, 2018-11-14 23:40:31.762414+00:00)


And then pulling:

In [34]:
deployments = cluster.pull_deployments()
deployments.jupyter_deployments

2018-11-15 00:21:10 INFO: Pulling deployments.
2018-11-15 00:21:13 INFO: Creating the ssh directory.
2018-11-15 00:21:21 INFO: Pulled Jupyter deployment: JupyterDeployment(53499 -> Node(p0097:54199, 2018-11-14 23:40:31.762414+00:00)


[JupyterDeployment(53499 -> Node(p0097:54199, 2018-11-14 23:40:31.762414+00:00)]

In [35]:
nb_2 = deployments.jupyter_deployments[0]
nb_2

JupyterDeployment(53499 -> Node(p0097:54199, 2018-11-14 23:40:31.762414+00:00)

In [36]:
nb_2.open_in_browser()

In [37]:
nb_2.cancel()
nb.cancel_local()

2018-11-15 00:21:26 INFO: Cancelling Jupyter deployment.


You can find more information on pushing and pulling deployments in the next section.

## Push and pull nodes

In order to work with Dask, you would usually need a notebook running on the cluster, as deployed above.

To access the allocated nodes from the cluster, you need to push their deployment first, same as the notebook deployment:

In [38]:
cluster.push_deployment(nodes)

2018-11-15 00:21:33 INFO: Pushing deployment: Nodes([Node(p0097:54199, 2018-11-14 23:40:31.762414+00:00),Node(p0100:33033, 2018-11-14 23:40:31.762414+00:00)], SlurmAllocation(job_id=14194860))


Then, you would pull the deployment on the cluster:

In [39]:
deployments = cluster.pull_deployments()
deployments

2018-11-15 00:21:38 INFO: Pulling deployments.
2018-11-15 00:21:41 INFO: Creating the ssh directory.
2018-11-15 00:21:49 INFO: Pulled allocation deployment: Nodes([Node(p0097:54199, 2018-11-14 23:40:31.762414+00:00),Node(p0100:33033, 2018-11-14 23:40:31.762414+00:00)], SlurmAllocation(job_id=14194860))
2018-11-15 00:21:49 INFO: Pulled Jupyter deployment: JupyterDeployment(53529 -> Node(p0097:54199, 2018-11-14 23:40:31.762414+00:00)
2018-11-15 00:21:57 INFO: Retried and failed: config.retries[Retry.VALIDATE_HTTP_TUNNEL].{count=3, seconds_between=2}
2018-11-15 00:21:57 INFO: Discarding a Jupyter deployment, because it is no longer functional: JupyterDeployment(53529 -> Node(p0097:54199, 2018-11-14 23:40:31.762414+00:00).


SynchronizedDeployments(nodes=1, jupyter_deployments=0)

In [40]:
nodes = deployments.nodes[0]
nodes

Nodes([Node(p0097:54199, 2018-11-14 23:40:31.762414+00:00),Node(p0100:33033, 2018-11-14 23:40:31.762414+00:00)], SlurmAllocation(job_id=14194860))

Essentially, this feature is intended for using an allocation in multiple notebooks at once.

Deployments are cleared automatically if they are expired or cancelled. They can also be cleared manually by  running:

In [41]:
cluster.clear_pushed_deployments()

2018-11-15 00:21:57 INFO: Clearing deployments.


## idact-notebook app

You can deploy nodes and notebook automatically using the following command:
```
idact-notebook
```
or:
```
python -m idact.notebook
```
Help message:

In [42]:
help_message = subprocess.getoutput(
    "cd .. && {python} -m idact.notebook --help".format(
        python=sys.executable))
print(help_message)

Usage: notebook.py [OPTIONS] CLUSTER_NAME

  A console script that executes a Jupyter Notebook instance on an allocated
  cluster node, and makes it accessible in the local browser.

  CLUSTER_NAME argument is the cluster name to execute the notebook on. It
  must already be present in the config file.

Options:
  -e, --environment TEXT  Environment path. Default: ~/.idact.conf or the
                          value of IDACT_CONFIG_PATH.
  --save-defaults         Save allocation parameters as defaults for next
                          time.
  --reset-defaults        Reset unspecified allocation parameters to defaults.
  --nodes INTEGER         Cluster node count. [Allocation parameter]. Jupyter
                          notebook will be deployed on the first node.
                          Default: 1.
  --cores INTEGER         CPU core count per node. [Allocation parameter].
                          Default: 1
  --memory-per-node TEXT  Memory per node. [Allocation parameter]. Default

For example, to deploy a notebook on a cluster with the same parameters as above, you could call:

```
python -m idact.notebook pro --save-defaults --environment notebooks/.idact-env --nodes 2 --cores 2 --memory-per-node 10GiB --walltime 0:20:00 --native-arg --partition plgrid-testing --native-arg --account intdata
```

The flag `--save-defaults` is optional, but it saves the allocation parameters: next time, the following will have the same effect:
```
python -m idact.notebook pro --environment notebooks/.idact-env
```
The `--environment` argument is optional if you use the default environment location.

The allocation and notebook the application deploys can be pulled from the cluster.

## Deploy Dask

You need to provide a list of Bash commands that will expose a Python distribution you want to deploy Dask with. It will likely be the same distribution as for the Jupyter notebook above.

Deploying Dask requires `dask`, `distributed`, and `bokeh` on the cluster. If you install `idact` with pip, they will be installed automatically.

In [43]:
cluster.config.setup_actions.dask = ['module load plgrid/tools/python-intel/3.6.2']
cluster.config.scratch = '$SCRATCH'
save_environment()

In [44]:
dd = deploy_dask(nodes)
dd

2018-11-15 00:21:59 INFO: Deploying Dask on 2 nodes.
2018-11-15 00:21:59 INFO: Connecting to p0097:54199 (1/2).
2018-11-15 00:21:59 INFO: Connecting to p0100:33033 (2/2).
2018-11-15 00:21:59 INFO: Deploying scheduler on the first node: p0097.
2018-11-15 00:22:12 INFO: Checking scheduler connectivity from p0097 (1/2).
2018-11-15 00:22:12 INFO: Checking scheduler connectivity from p0100 (2/2).
2018-11-15 00:22:12 INFO: Deploying workers.
2018-11-15 00:22:12 INFO: Deploying worker 1/2.
2018-11-15 00:22:23 INFO: Deploying worker 2/2.
2018-11-15 00:22:33 INFO: Validating worker 1/2.
2018-11-15 00:22:33 INFO: Validating worker 2/2.


DaskDeployment(scheduler=tcp://localhost:53541/tcp://172.20.64.97:56230, workers=2)

In [45]:
nodes[0].resources.memory_usage

GiB(0.2905616760253906)

Get Dask client:

In [46]:
client = dd.get_client()
client

<Client: scheduler='tcp://172.20.64.97:56230' processes=2 cores=4>

In [47]:
nodes[0].resources.cpu_usage

9.0

Computation will work only if Python and library versions match:

In [48]:
#x = client.submit(lambda: value + 1, 10)
#x.result() == 11

Diagnostics servers are tunnelled:

In [49]:
dd.diagnostics.addresses

['http://localhost:45934/status',
 'http://localhost:53554/main',
 'http://localhost:53562/main']

To open diagnostics servers in new tabs:

In [50]:
dd.diagnostics.open_all()

In [51]:
dd.cancel()

2018-11-15 00:23:49 INFO: Cancelling worker deployment on p0100.
2018-11-15 00:24:00 INFO: Cancelling worker deployment on p0097.
2018-11-15 00:24:07 INFO: Cancelling scheduler deployment on p0097.


## Adjust timeouts

Sometimes a timeout occurs during a deployment, and may even cause it to fail. 

If you find this to happen too often, you may need to adjust the timeouts for your cluster.

In order to do that, copy the retry name from the info message preceding the failure that looks similar to this:

```
2018-11-12 22:14:00 INFO: Retried and failed: config.retries[Retry.PORT_INFO].{count=5, seconds_between=5}
```

First, you can look up what the current retry config is:

In [52]:
cluster.config.retries[Retry.PORT_INFO]

RetryConfig(count=5, seconds_between=5)

And adjust the retry count and/or seconds between retries:

In [53]:
cluster.config.retries[Retry.PORT_INFO] = set_retry(count=6,
                                                    seconds_between=10)
cluster.config.retries[Retry.PORT_INFO]

RetryConfig(count=6, seconds_between=10)

Alternatively:

In [54]:
cluster.config.retries[Retry.PORT_INFO].count = 6
cluster.config.retries[Retry.PORT_INFO].seconds_between = 10
cluster.config.retries[Retry.PORT_INFO]

RetryConfig(count=6, seconds_between=10)

### Defaults

In [55]:
get_default_retries()

{<Retry.PORT_INFO: 0>: RetryConfig(count=5, seconds_between=5),
 <Retry.JUPYTER_JSON: 1>: RetryConfig(count=5, seconds_between=3),
 <Retry.SCHEDULER_CONNECT: 2>: RetryConfig(count=5, seconds_between=2),
 <Retry.DASK_NODE_CONNECT: 3>: RetryConfig(count=3, seconds_between=5),
 <Retry.DEPLOY_DASK_SCHEDULER: 4>: RetryConfig(count=3, seconds_between=5),
 <Retry.DEPLOY_DASK_WORKER: 5>: RetryConfig(count=3, seconds_between=5),
 <Retry.GET_SCHEDULER_ADDRESS: 6>: RetryConfig(count=5, seconds_between=5),
 <Retry.CHECK_WORKER_STARTED: 7>: RetryConfig(count=5, seconds_between=5),
 <Retry.CANCEL_DEPLOYMENT: 8>: RetryConfig(count=5, seconds_between=1),
 <Retry.SQUEUE_AFTER_SBATCH: 9>: RetryConfig(count=3, seconds_between=3),
 <Retry.OPEN_TUNNEL: 10>: RetryConfig(count=3, seconds_between=5),
 <Retry.VALIDATE_HTTP_TUNNEL: 11>: RetryConfig(count=3, seconds_between=2)}

## Close

In [56]:
nodes.running()

True

In [57]:
nodes.cancel()

2018-11-15 00:24:31 INFO: Cancelling job 14194860.


In [58]:
nodes.running()

False

In [59]:
node.run('squeue')

'JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)\n          14194860 plgrid-te     wrap plggarst CG       4:00      2 p[0097,0100]'

## Push and pull the environment

When working on a cluster, it may be useful to synchronize idact config with the local machine. Pushing the environment will merge the local environment into the remote environment.

In [60]:
push_environment(cluster)

2018-11-15 00:24:35 INFO: Pushing the environment to cluster.
2018-11-15 00:24:37 ERROR: Failure: Getting file from node pro.cyfronet.pl: /net/people/plggarstka/.idact.conf
2018-11-15 00:24:37 ERROR: Failure: Deserializing the environment from cluster.
2018-11-15 00:24:37 INFO: Remote environment is missing, current environment will be copied to cluster.


You will be able to use that environment when working on the remote notebook.

In [61]:
print(node.run('cat ~/.idact.conf'))

{
    "clusters": {
        "pro": {
            "auth": "PUBLIC_KEY",
            "disableSshd": false,
            "host": "pro.cyfronet.pl",
            "installKey": true,
            "key": null,
            "notebookDefaults": {},
            "port": 22,
            "retries": {
                "CANCEL_DEPLOYMENT": {
                    "count": 5,
                    "secondsBetween": 1
                },
                "CHECK_WORKER_STARTED": {
                    "count": 5,
                    "secondsBetween": 5
                },
                "DASK_NODE_CONNECT": {
                    "count": 3,
                    "secondsBetween": 5
                },
                "DEPLOY_DASK_SCHEDULER": {
                    "count": 3,
                    "secondsBetween": 5
                },
                "DEPLOY_DASK_WORKER": {
                    "count": 3,
                    "secondsBetween": 5
                },
                "GET_SCHEDULER_ADDRESS": {
             

The reverse operation is pulling the environment, which merges the remote environment into the local environment. Machine-specific information like the private key path is skipped when pushing or pulling.

In [62]:
pull_environment(cluster)

2018-11-15 00:24:50 INFO: Pulling the environment from cluster.


You can remove it if you don't need it for now:

In [63]:
# node.run('rm -v ~/.idact.conf')

## Remove cluster

A cluster can be removed from the environment.

In [64]:
add_cluster(name='fake',
            user='fakeuser',
            host='fakehost',
            port=2222)

2018-11-15 00:24:53 INFO: No auth method specified, defaulting to password-based.


Cluster(fakehost, 2222, fakeuser, auth=AuthMethod.ASK, key=None, install_key=True, disable_sshd=False)

In [65]:
show_clusters()

{'pro': Cluster(pro.cyfronet.pl, 22, plggarstka, auth=AuthMethod.PUBLIC_KEY, key='C:\\Users\\Maciej/.ssh\\id_rsa_j7', install_key=False, disable_sshd=False),
 'fake': Cluster(fakehost, 2222, fakeuser, auth=AuthMethod.ASK, key=None, install_key=True, disable_sshd=False)}

In [66]:
remove_cluster('fake')

In [67]:
show_clusters()

{'pro': Cluster(pro.cyfronet.pl, 22, plggarstka, auth=AuthMethod.PUBLIC_KEY, key='C:\\Users\\Maciej/.ssh\\id_rsa_j7', install_key=False, disable_sshd=False)}